In [17]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [18]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

In [50]:
#df = pd.DataFrame(list(analyseCol.find({'soort': 'Stelling'})))
df = pd.DataFrame(list(analyseColClean.find({'soort': 'Put'})))
df.drop(['ID', 'index', 'level_0'], inplace=True, errors='ignore')
df.reset_index(inplace=True)
df['ID'] = df['index']
df

,index,_id,brondata,projectcd,putnr,soort,key,key_project,ID
0,0,6201144fbba1b3fdae51bef9,"{'_id': 6201144fbba1b3fdae51bef9, 'PUT': 1, 'P...",DC154,1,Put,PDC154P1,PDC154,0
1,1,6201144fbba1b3fdae51bef8,"{'_id': 6201144fbba1b3fdae51bef8, 'PUT': 2, 'P...",DC154,2,Put,PDC154P2,PDC154,1
2,2,6201146088473abe7f90507d,"{'_id': 6201146088473abe7f90507d, 'PUT': 1, 'P...",DC179,1,Put,PDC179P1,PDC179,2
3,3,6201146088473abe7f90507e,"{'_id': 6201146088473abe7f90507e, 'PUT': 2, 'P...",DC179,2,Put,PDC179P2,PDC179,3
4,4,6201146088473abe7f90507f,"{'_id': 6201146088473abe7f90507f, 'PUT': 3, 'P...",DC179,3,Put,PDC179P3,PDC179,4
5,5,6201146088473abe7f905080,"{'_id': 6201146088473abe7f905080, 'PUT': 4, 'P...",DC179,4,Put,PDC179P4,PDC179,5
6,6,6201146088473abe7f905081,"{'_id': 6201146088473abe7f905081, 'PUT': 5, 'P...",DC179,5,Put,PDC179P5,PDC179,6
7,7,6201146088473abe7f905082,"{'_id': 6201146088473abe7f905082, 'PUT': 6, 'P...",DC179,6,Put,PDC179P6,PDC179,7
8,8,6201146088473abe7f905083,"{'_id': 6201146088473abe7f905083, 'PUT': 7, 'P...",DC179,7,Put,PDC179P7,PDC179,8
9,9,6201146088473abe7f905084,"{'_id': 6201146088473abe7f905084, 'PUT': 8, 'P...",DC179,8,Put,PDC179P8,PDC179,9


In [63]:
pipelines = [[{'$match': {'$and': [{'putnr': {'$exists': True}}, {'projectcd': {'$exists': True}}]}}, {'$group': {'_id': {'projectcd': '$projectcd', 'putnr': '$putnr'}}}, {'$unwind': '$_id'}, {'$project': {'_id': 0, 'projectcd': '$_id.projectcd', 'putnr': '$_id.putnr'}}, {'$addFields': {'brondata.table': 'generated', 'soort': 'Put'}}, {'$match': {'soort': 'Put'}}, {'$addFields': {'key': {'$concat': ['P', '$projectcd', 'P', {'$toString': '$putnr'}]}}}, {'$addFields': {'key_project': {'$concat': ['P', '$projectcd']}}}]]

df_gen = pd.DataFrame(list(analyseCol.aggregate(pipelines[0])))
df_orig = pd.DataFrame(list(analyseColClean.find({'soort': 'Put'})))

df_all = df_gen.merge(df_orig, on=['key', 'soort'], how='left', indicator=True, suffixes=[None, "_orig"])
df_all[df_all._merge == 'left_only'][df_gen.columns]

,projectcd,putnr,brondata,soort,key,key_project
1,DB34,9.0,{'table': 'generated'},Put,PDB34P9,PDB34
2,DC36,1.0,{'table': 'generated'},Put,PDC36P1,PDC36
4,DC36,2.0,{'table': 'generated'},Put,PDC36P2,PDC36
5,DC97,3.0,{'table': 'generated'},Put,PDC97P3,PDC97
6,DC72,0.0,{'table': 'generated'},Put,PDC72P0,PDC72
8,DC97,1.0,{'table': 'generated'},Put,PDC97P1,PDC97
10,DB34,11.0,{'table': 'generated'},Put,PDB34P11,PDB34
11,DC97,4.0,{'table': 'generated'},Put,PDC97P4,PDC97
12,DC36,0.0,{'table': 'generated'},Put,PDC36P0,PDC36
13,DB34,2.0,{'table': 'generated'},Put,PDB34P2,PDB34


In [65]:
df_orig = pd.DataFrame(list(analyseColClean.find({'soort': 'Vlak'})))
df_orig

,_id,brondata,projectcd,soort,key_project
0,6201909497c275f105f4c107,"{'_id': 6201909497c275f105f4c107, 'PUT': 2, 'V...",DC154,Vlak,PDC154
1,6201909497c275f105f4c108,"{'_id': 6201909497c275f105f4c108, 'PUT': 2, 'V...",DC154,Vlak,PDC154
2,6201909497c275f105f4c109,"{'_id': 6201909497c275f105f4c109, 'PUT': 1, 'V...",DC154,Vlak,PDC154
3,6201909497c275f105f4c10a,"{'_id': 6201909497c275f105f4c10a, 'PUT': 1, 'V...",DC154,Vlak,PDC154
4,6201909497c275f105f4c10b,"{'_id': 6201909497c275f105f4c10b, 'PUT': 2, 'V...",DC154,Vlak,PDC154
5,6201909e9b2b04f87217318a,"{'_id': 6201909e9b2b04f87217318a, 'PUT': 1, 'V...",DC179,Vlak,PDC179
6,6201909e9b2b04f87217318b,"{'_id': 6201909e9b2b04f87217318b, 'PUT': 1, 'V...",DC179,Vlak,PDC179
7,6201909e9b2b04f87217318c,"{'_id': 6201909e9b2b04f87217318c, 'PUT': 2, 'V...",DC179,Vlak,PDC179
8,6201909e9b2b04f87217318d,"{'_id': 6201909e9b2b04f87217318d, 'PUT': 2, 'V...",DC179,Vlak,PDC179
9,6201909e9b2b04f87217318e,"{'_id': 6201909e9b2b04f87217318e, 'PUT': 1, 'V...",DC179,Vlak,PDC179
